In [1]:
import numpy as np
from itertools import combinations, product
from scipy.linalg import eigh

In [2]:
h1e = np.load("1e.npy")
# (ij|kl) in chemists' notation
h2e = np.load("2e.npy")

# scf coefficnets, the 1st index is atomic basis, the 2nd index is the molecular orbital
# calculated from SimpleSCF.ipynb
scf_c = np.load("scf.npy")

# basis transformation to canonical HF orbital basis
h1e = scf_c.T @ h1e @ scf_c
for i in range(4):
    h2e = np.tensordot(h2e, scf_c, axes=1).transpose(3, 0, 1, 2)

In [3]:
# nuclear repulsion energy
e_nuc = 203.030749104

# declaration of the spins
alpha = "alpha"
beta  = "beta"

In [4]:
# Table 2.3 and 2.4
def elem_case1(active_alpha_list, active_beta_list):
    alpha_list = inactive_list + active_alpha_list
    beta_list  = inactive_list + active_beta_list
    # <O_1>
    o1 = h1e[alpha_list, alpha_list].sum() + h1e[beta_list, beta_list].sum()
    # <O_2>
    # alpha-alpha interaction
    m, n = np.array(list(product(alpha_list, alpha_list))).T
    o2 = (h2e[m, m, n, n] - h2e[m, n, n, m]).sum()
    # alpha-beta interaction
    m, n = np.array(list(product(alpha_list, beta_list))).T
    o2 += h2e[m, m, n, n].sum()
    # beta-alpha interaction
    m, n = np.array(list(product(beta_list, alpha_list))).T
    o2 += h2e[m, m, n, n].sum()
    # beta-beta interaction
    m, n = np.array(list(product(beta_list, beta_list))).T
    o2 += (h2e[m, m, n, n] - h2e[m, n, n, m]).sum()
    o2 /= 2
    return o1 + o2

In [5]:
# Table 2.3 and 2.4
def elem_case2(m, p, active_samespin_list, active_diffspin_list):
    # <O_1>
    o1 = h1e[m, p]
    # <O_2>
    # orbitals with the same spins。 Note that m and p must have the same spin due to spin conservation
    samespin_list = inactive_list + active_samespin_list
    # orbitals with the different (opposite) spins.
    diffspin_list = inactive_list + active_diffspin_list
    # interaction with the same spins. 
    o2 = (h2e[m, p, samespin_list, samespin_list] - h2e[m, samespin_list, samespin_list, p]).sum()
    # interaction with the different (opposite) spins
    o2 += h2e[m, p, diffspin_list, diffspin_list].sum()
    return o1 + o2

In [6]:
# Table 2.3 and 2.4
def elem_case3(m, mspin, n, nspin, p, pspin, q, qspin):
    # m, n, p, q has to be ordered
    assert m <= n and p <= q
    # <O_1> = 0
    o1 = 0
    # <O_2>
    o2 = 0
    if mspin == pspin and nspin == qspin:
        o2 += h2e[m, p, n, q]
    if mspin == qspin and nspin == pspin:
        o2 -= h2e[m, q, n, p]
    return o1 + o2

In [7]:
def count_order(m, mspin, active_alpha_list, active_beta_list):
    # count the number of operatorions required to move m to the last position of the orbitals
    # For spin-orbitals with the same space-orbital, first alpha then beta
    if mspin == alpha:
        return (m < np.array(active_alpha_list)).sum() + (m <= np.array(active_beta_list)).sum()
    else:
        return (m < np.array(active_alpha_list)).sum() + (m <  np.array(active_beta_list)).sum()

In [8]:
def get_h_ci(configurations):
    # construct CI Hamiltonian
    h_ci = np.zeros((len(configurations), len(configurations)))
    for i, (active_alpha_list1, active_beta_list1) in enumerate(configurations):
        for j, (active_alpha_list2, active_beta_list2) in enumerate(configurations):
            # preparations
            active_alpha_set1, active_beta_set1, active_alpha_set2, active_beta_set2 \
              = map(set, [active_alpha_list1, active_beta_list1, active_alpha_list2, active_beta_list2])
            unique_alpha = active_alpha_set1 | active_alpha_set2
            diff_alpha = unique_alpha - (active_alpha_set1 & active_alpha_set2)
            unique_beta  = active_beta_set1  | active_beta_set2
            diff_beta = unique_beta  -  (active_beta_set1  & active_beta_set2)
            total_n_unique = len(unique_alpha) + len(unique_beta)
            
            # case 1, same state
            if total_n_unique == active_n_e:
                h_ci[i, j] = elem_case1(active_alpha_list1, active_beta_list1)
                continue
            # case 2, differ by 1 orbital
            if total_n_unique == active_n_e + 1:
                if n_active_alpha < len(unique_alpha):
                    # different spin-orbital is alpha
                    m = (unique_alpha - active_alpha_set2).pop()
                    p = (unique_alpha - active_alpha_set1).pop()
                    mspin = pspin = alpha
                    elem = elem_case2(m, p, active_alpha_list1, active_beta_list1)
                else:
                    # different spin-orbital is beta
                    m = (unique_beta  - active_beta_set2).pop()
                    p = (unique_beta  - active_beta_set1).pop()
                    mspin = pspin = beta
                    elem = elem_case2(m, p, active_beta_list1, active_alpha_list1)
                # move the different orbital to the last position of the orbitals
                sign = (-1) ** (count_order(m, mspin, active_alpha_list1, active_beta_list1) + count_order(p, pspin, active_alpha_list2, active_beta_list2))
                h_ci[i, j] = sign * elem
                continue
            # case 3, differ by 2 orbitals
            if total_n_unique == active_n_e + 2:
                if diff_alpha and not diff_beta:
                    # different orbitals are all alpha
                    m, n = sorted(unique_alpha - active_alpha_set2)
                    p, q = sorted(unique_alpha - active_alpha_set1)
                    mspin = nspin = pspin = qspin = alpha
                elif not diff_alpha and diff_beta:
                    # different orbitals are all beta
                    m, n = sorted(unique_beta - active_beta_set2)
                    p, q = sorted(unique_beta - active_beta_set1)
                    mspin = nspin = pspin = qspin = beta
                elif diff_alpha and diff_beta:
                    # different orbtals half alpha half beta
                    m = (unique_alpha - active_alpha_set2).pop()
                    n = (unique_beta  - active_beta_set2 ).pop()
                    mspin, nspin = alpha, beta
                    if n < m:
                        m, n = n, m
                        mspin, nspin = nspin, mspin
                    p = (unique_alpha - active_alpha_set1).pop()
                    q = (unique_beta  - active_beta_set1 ).pop()
                    pspin, qspin = alpha, beta
                    if q < p:
                        p, q = q, p
                        pspin, qspin = qspin, pspin
                else:
                    assert False
                elem = elem_case3(m, mspin, n, nspin, p, pspin,  q, qspin)
                # move the different orbitals to the last two positions of the orbitals
                total_sign_count = count_order(m, mspin, active_alpha_list1, active_beta_list1) \
                                 + count_order(n, nspin, active_alpha_list1, active_beta_list1) \
                                 + count_order(p, pspin, active_alpha_list2, active_beta_list2) \
                                 + count_order(q, qspin, active_alpha_list2, active_beta_list2)
                # `-2` is because the last two positions are occupied. One less operation for each of K and L.
                sign = (-1) ** (total_sign_count - 2)
                h_ci[i, j] = sign * elem
    return h_ci

In [9]:
active_space = [16, 19, 20, 21, 22, 23]
inactive_list = list(range(16)) + [17, 18]
n_active_alpha = 3
n_active_beta  = 3
alpha_combinations = list(map(list, combinations(active_space, n_active_alpha)))
beta_combinations  = list(map(list, combinations(active_space, n_active_beta)))
configurations = list(product(alpha_combinations, beta_combinations))
active_n_e = n_active_alpha + n_active_beta
h_ci = get_h_ci(configurations)
np.linalg.eigvalsh(h_ci)[:2] + e_nuc

array([-227.99524406, -227.84865301])

In [10]:
active_space = [16, 19, 20, 21, 22, 23]
inactive_list = list(range(16)) + [17, 18]
n_active_alpha = 4
n_active_beta  = 2
alpha_combinations = list(map(list, combinations(active_space, n_active_alpha)))
beta_combinations  = list(map(list, combinations(active_space, n_active_beta)))
configurations = list(product(alpha_combinations, beta_combinations))
active_n_e = n_active_alpha + n_active_beta
h_ci = get_h_ci(configurations)
np.linalg.eigvalsh(h_ci)[:2] + e_nuc

array([-227.84865301, -227.78892865])